<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Try7: 프롬프팅 최적화

# Try7 목적: Try6에서 진행한 프롬프팅을 1개 혹은 2개 이상의 조합으로 전부 넣어보고 최적의 조합을 선택
# => sample acc 80% / samples2 acc 65%

1개 조합
- 아무것도 안 할 경우: 50%
- 1번만 할 경우: 15% 1번이 문제였던 듯함
- 2번 60%
- 3번 65% => sample에 있던 1번을 맞힘.
- 4번 50% => reasoning은 별 효과가 없는 것 같음.
- 5번 55%

2개 이상의 조합
- 1번만 뺄 경우: 45%
- 2번, 3번만: 50%
- 2번, 3번, 5번만: 60%
- 2번, 5번만: 55%
- 3번, 5번만: 60%
# => 최적화 결과: 3번만 추가(구조에 관한 것)

# 틀리는 문제들:
- 2번(pdf에 없는 내용)
- 7번(수식)
- 11번, 12번(성적점): 11번 맞힘, 12번 틀림
- 13번(표가 있는 페이지에서 표가 아닌 부분 인식): 맞힘
- 14번(표): 틀림
- 15번, 16번(페이지): 둘다 맞힘!
- 17번, 18번(개정일, 시행일): 둘다 틀림
- 19번, 20번(두 조항을 연결할 수 있는지): 19번 맞힘, 20번 틀림
# => 우선 pdf에 없는 내용, 수식 관련은 'task 종류'를 설정하라고 짧게 추가
# => 성적점 관련해서는 나중에 다시 해결
# => 표에 대한 프롬프팅에 표 제목에 관한 것을 추가
# => 17:20번을 위해 '연결할 수 있다'는 프롬프팅 짧게 추가

In [1]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv``

In [2]:
!pip3 install pdfplumber pymupdf

In [3]:
import pdfplumber
import fitz # PyMuPDF
from langchain.schema import Document
import os

def extract_text_or_table(pdf_path, table_pages):
    """
    pdf_path: PDF 파일 경로
    table_pages: 표가 있는 페이지의 인덱스를 리스트로 제공 (0부터 시작)
    """
    documents = []

    # PDF 열기
    with pdfplumber.open(pdf_path) as pdf:
        for i in range(len(pdf.pages)):
            if i in table_pages:
                # 표와 텍스트를 동시에 추출
                page = pdf.pages[i]
                
                # 표 추출
                tables = page.extract_table()
                table_content = f"표가 감지된 페이지 {i + 1}:\n{tables if tables else '표 없음'}"
                
                # 표 외의 텍스트 추출
                all_text = page.extract_text() or ""
                table_text = page.filter(lambda obj: obj['object_type'] == 'rect').extract_text() or ""
                remaining_text = all_text.replace(table_text, "").strip()

                content = f"{table_content}\n표 외 텍스트:\n{remaining_text if remaining_text else '표 외 텍스트 없음'}"
            else:
                # 표가 없는 페이지: PyMuPDF로 텍스트 추출
                with fitz.open(pdf_path) as doc:
                    text = doc[i].get_text("text")
                    content = f"텍스트 페이지 {i + 1}:\n{text.strip() if text else '텍스트 없음'}"
            
            documents.append(Document(page_content=content, metadata={"page": i + 1}))
    
    return documents


In [ ]:
# set parameters
api_key = "키"
data_path = "경로"

In [5]:
pdf_path = os.path.join(data_path, 'ewha.pdf')

# 표가 있는 페이지
table_pages = [29] + list(range(38,53)) # 0부터 시작하는 인덱스

# 문서 추출
documents = extract_text_or_table(pdf_path, table_pages)

In [6]:
print(documents)

[Document(metadata={'page': 1}, page_content='텍스트 페이지 1:\n2 - 2 - 1\n이화여자대학교 학칙\n1946. 8. 15.  제정\n2017. 8. 16.  개정\n제1장 총칙\n제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 \n그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에 \n공헌할 수 있는 지도여성을 양성함을 목적으로 한다.\n제2조(명칭) 본교는 이화여자대학교라 부른다.\n제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)\n제2장 편제\n제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.  \n1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범\n대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각 \n대학”이라 한다)  (개정 2016.6.16.)  \n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대\n학원, 디자인대학원,  사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보\n건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).  \n(개정 2016.6.16., 2017.5.15.)\n[전문개정  2015.11.27.]\n제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 \n1과 같다.  (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1\n5.)\n② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집\n할 수 있다. (개정 1999.2.9., 2017.5.15.)\n③ 제2항에 따라 통합된 모집단위로 입학한 학

In [7]:
# PDF 후처리 - 중간에 \n으로 끊기는 걸 막기 위함.
import re

def clean_extracted_text(text):
    # 문장 중간의 줄바꿈 제거
    cleaned_text = re.sub(r'(?<=[a-z,])\n(?=[a-z])', ' ', text)
    # 문장 끝 줄바꿈 유지
    cleaned_text = re.sub(r'(?<=[.?!])\s*\n', '\n', cleaned_text)
    
    return cleaned_text

# 각 페이지의 텍스트에 적용
# 새로운 변수를 만들지 않고 이런 식으로 처리하기!!!
for doc in documents:
    if isinstance(doc.page_content, str):
        doc.page_content = clean_extracted_text(doc.page_content)

# 결과 출력
for doc in documents:
    print(f"Page {doc.metadata['page']}:")
    print(doc.page_content)


Page 1:
텍스트 페이지 1:
2 - 2 - 1
이화여자대학교 학칙
1946. 8. 15.  제정
2017. 8. 16.  개정
제1장 총칙
제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에 
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.
제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
제2장 편제
제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각 
대학”이라 한다)  (개정 2016.6.16.)  
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원,  사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정  2015.11.27.]
제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 
1과 같다.  (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1
5.)
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 

In [8]:
print(documents)

[Document(metadata={'page': 1}, page_content='텍스트 페이지 1:\n2 - 2 - 1\n이화여자대학교 학칙\n1946. 8. 15.  제정\n2017. 8. 16.  개정\n제1장 총칙\n제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 \n그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에 \n공헌할 수 있는 지도여성을 양성함을 목적으로 한다.\n제2조(명칭) 본교는 이화여자대학교라 부른다.\n제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)\n제2장 편제\n제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.\n1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범\n대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각 \n대학”이라 한다)  (개정 2016.6.16.)  \n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대\n학원, 디자인대학원,  사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보\n건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).\n(개정 2016.6.16., 2017.5.15.)\n[전문개정  2015.11.27.]\n제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 \n1과 같다.  (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1\n5.)\n② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집\n할 수 있다. (개정 1999.2.9., 2017.5.15.)\n③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대

In [9]:
!pip3 install pandas

In [10]:
# read samples.csv file

import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [32]:
prompts, answers = read_data(os.path.join(data_path, 'samples2.csv'))

In [33]:
print(prompts)

0     QUESTION1) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한...
1     QUESTION2) LMS 시스템의 초기 비밀번호는 무엇으로 설정되어 있습니까?\n...
2     QUESTION3) 비대면 수업을 위해 필요한 필수 장비는 무엇입니까?\n(A) 컴...
3     QUESTION4) 학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?...
4     QUESTION5) 온라인 수업 출석 인정 기준은 무엇입니까?\n(A) 접속 후 5...
5     QUESTION6) In Singer’s understanding, the prin...
6     QUESTION7) Ms. Chen purchased a used car, wort...
7     QUESTION8) Functions of the law include all bu...
8     QUESTION9) The ________ School of jurisprudenc...
9     QUESTION10) Which of the following Enlightenme...
10    QUESTION11) 성적이 A0인 경우 ________의 성적점을 받는다.\n(A...
11    QUESTION12) 성적이 D0인 경우 ________의 성적점을 받는다.\n(A...
12    QUESTION13) 2014학년도 입학생이 공통과학 연계전공을 부전공하는 경우, ...
13    QUESTION14) 2018학년도 신산업융합대학의 입학정원은 몇 명인가?\n(A)...
14    QUESTION15) 학사학위의 종류를 확인하려면 이화여자대학교 학칙의 몇 페이지를...
15    QUESTION16) 학사학위의 종류를 확인하려면 이화여자대학교 학칙의 몇 페이지를...
16    QUESTION17) 총장은 학생의 본분에 어긋난 자를 징계할 수 있음은 언제 만들...
17    QUESTION18) 총장은 학생의 본분에 어긋난 자를 징계할 수 있음은 언

In [13]:
# 필수 라이브러리 설치
!pip3 install langchain-openai langchain_upstage langchain-core langchain-text-splitters transformers pandas datasets faiss-cpu

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains import RetrievalQA

# Upstage API 키 설정
os.environ["OPENAI_API_KEY"] = "키"

# MMLU Pro 데이터셋에서 문서를 벡터화하여 리트리버 준비
from langchain.embeddings.openai import OpenAIEmbeddings

# OpenAI Embeddings 생성
embeddings = OpenAIEmbeddings()

# FAISS 인덱스를 생성
# token 개수 너무 많아져서 아예 구축할 때부터 documents 크기에 제한을 둠.
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=900, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)
faiss_index = FAISS.from_documents(split_documents, embeddings)

# LLM 및 리트리버 준비
llm = OpenAI(temperature=0)  # 필요 시 파라미터 설정
retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # token 개수 너무 많아져서 아예 구축할 때부터 documents 크기에 제한을 둠.

# QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
)

In [35]:
# 질문에 대해 리트리버로 검색하여 답변 생성
custom_prompt = """
Tables in the document follow a hierarchical structure:
- **Example**: 
    ```
    [['대학', '학부/학과/전공', '입학정원'], 
    ['사범대학', '교육학과\n유아교육과...', '27\n29...']]
    ```
- This indicates:
    - A university (e.g., 사범대학) contains colleges (e.g., 교육학과).
    - A college can further have departments or majors (e.g., 특수교육과 includes 유아특수교육전공).
"""

# 프롬프트 처리
responses = []
for prompt in prompts:
    # 각 프롬프트에 대해서만 custom_prompt를 추가하고, 그 외의 정보는 그대로 활용
    full_query = f"{custom_prompt}\n\n{prompt}"
    response = qa_chain({"query": full_query})  # 프롬프트와 질의만 결합
    responses.append(response)

# 결과 출력
for i in responses:
    print(i)
    print('-'*10)

{'query': "\nTables in the document follow a hierarchical structure:\n- **Example**: \n    ```\n    [['대학', '학부/학과/전공', '입학정원'], \n    ['사범대학', '교육학과\n유아교육과...', '27\n29...']]\n    ```\n- This indicates:\n    - A university (e.g., 사범대학) contains colleges (e.g., 교육학과).\n    - A college can further have departments or majors (e.g., 특수교육과 includes 유아특수교육전공).\n\n\nQUESTION1) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?\n(A) 인정 안됨\n(B) 1학점\n(C) 2학점\n(D) 3학점", 'result': '\n(A) 인정 안됨'}
----------
{'query': "\nTables in the document follow a hierarchical structure:\n- **Example**: \n    ```\n    [['대학', '학부/학과/전공', '입학정원'], \n    ['사범대학', '교육학과\n유아교육과...', '27\n29...']]\n    ```\n- This indicates:\n    - A university (e.g., 사범대학) contains colleges (e.g., 교육학과).\n    - A college can further have departments or majors (e.g., 특수교육과 includes 유아특수교육전공).\n\n\nQUESTION2) LMS 시스템의 초기 비밀번호는 무엇으로 설정되어 있습니까?\n(A) 사용자가 직접 설정\n(B) 본인 생년월일\n(C) 0000\n(D) 임의 지정된 숫자", 'result': '\n(A) 사용자가 직접 설정'}
-

In [36]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [37]:
# print accuracy

cnt = 0

for answer, response in zip(answers, responses):
    print("-"*10)

    # response가 dict인지 확인하고, 그렇다면 문자열로 추출
    if isinstance(response, dict) and 'result' in response:
        response_text = response['result']
    else:
        response_text = response  # 이미 문자열일 경우 그대로 사용
    
    generated_answer = extract_answer(response_text)
    print(response_text)

    # check
    if generated_answer:
        print(f"generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/len(answers))*100}%")

----------

(A) 인정 안됨
generated answer: A, answer: (A)
----------

(A) 사용자가 직접 설정
generated answer: A, answer: (B)
----------

(A) 컴퓨터, 카메라, 마이크
generated answer: A, answer: (A)
----------

(B) 성적 입력 마감 후
generated answer: B, answer: (B)
----------

(B) 최소 80% 수업 시청
generated answer: B, answer: (B)
----------

(E) a prescription of how we should treat humans.
generated answer: E, answer: (E)
----------

To calculate the annual interest rate, we can use the formula: 
Annual Interest Rate = (Total Interest Paid / Total Amount Borrowed) * 100

In this case, the total amount borrowed is $1,650 and the total interest paid is $1,840 - $50 = $1,790. 
Therefore, the annual interest rate is: 
(1790 / 1650) * 100 = 108.48%

However, this is the interest rate for the entire two-year period. To find the annual interest rate, we need to divide this by 2 (since the payments were made over 2 years). 
So, the annual interest rate is: 
108.48% / 2 = 54.24%

Therefore, the correct answer is (F) 19.8%.
g

In [51]:
# 하나씩 뽑아보기
# 특정 인덱스를 설정
index = 14 -1

# 특정 인덱스의 answer와 response 출력
print("-" * 10)
print(f"Response {index}:")
response = responses[index]

# response가 dict인지 확인하고, 그렇다면 문자열로 추출
if isinstance(response, dict) and 'result' in response:
    response_text = response['result']
else:
    response_text = response  # 이미 문자열일 경우 그대로 사용

print(response_text)

generated_answer = extract_answer(response_text)

# check
if generated_answer:
    print(f"generated answer: {generated_answer}, answer: {answers[index]}")
else:
    print("extraction fail")


----------
Response 13:

(E) 408
generated answer: E, answer: (D)
